In [1]:
import torch
import torch.distributions as tdist
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (20,10)


from datetime import datetime
import numpy as np
from numpy import log, sqrt, exp
import imageio
from scipy.io import FortranFile
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline

In [2]:
f = FortranFile('C:/Users/Lawrence Huang/Desktop/Research/z3taured.dat', 'r')
nlos=int(np.asscalar(f.read_ints()))
print("nlos = %d" %nlos)
npix=int(np.asscalar(f.read_ints()))
zred=np.asscalar(f.read_record('f4'))
blenkms=np.asscalar(f.read_record('f4'))
blen=np.asscalar(f.read_record('f4'))*0.001  #back into mpc/h
print(nlos,npix,zred,blenkms,blen)
taured=[]
nstep=1 #skipping through in steps of 1
for i in range(0,nlos,nstep):       
    tauredin=f.read_record('f4')
    taured.extend(tauredin)
f.close()
print('len(taured)=',len(taured))

nlos=int(nlos/nstep)

print ('nlos,npix=',nlos,npix)

taured=np.array(taured)

taured=np.reshape(taured,(nlos,1,npix))
alternativeTaured = np.reshape(taured, (256,256,1,npix))

print('shape of taured=',taured.shape)
# print(alternativeTaured[0,0] == taured[0])
# print(alternativeTaured[1,1] == taured[257])
# print(taured[0,0])

nlos = 65536
65536 512 3.0 42807.71484375 400.0
len(taured)= 33554432
nlos,npix= 65536 512
dasme
(33554432,)
shape of taured= (65536, 1, 512)
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True 

In [3]:
tauTest = np.reshape(alternativeTaured[0:114,0:114],(114*114,1,512))
tauValidate = np.reshape(alternativeTaured[-114:,-114:],(114*114,1,512))
tauTrain = np.concatenate((
               np.reshape(alternativeTaured[:114,114:],(114*(256-114),1,512)), \
               np.reshape(alternativeTaured[114:-114],(256*(256-2*114),1,512)), \
               np.reshape(alternativeTaured[-114:,:-114],(114*(256-114),1,512))))


In [18]:
mintaured=taured.min()
maxtaured=taured.max()
taured[taured > 0.5e9] = -1.e10

tauTest[tauTest > 0.5e9] = -1.e10
tauValidate[tauValidate > 0.5e9] = -1.e10
tauTrain[tauTrain > 0.5e9] = -1.e10

maxtaured=taured.max()
print (maxtaured)
# set to realistic max:

taured[taured < -0.5e9] = maxtaured
tauTest[tauTest < -0.5e9] = maxtaured
tauValidate[tauValidate < -0.5e9] = maxtaured
tauTrain[tauTrain < -0.5e9] = maxtaured

#SCALING

# taured=(((taured-mintaured)/(maxtaured-mintaured))*0.98)+0.01
# tauTest=(((tauTest-mintaured)/(maxtaured-mintaured))*0.98)+0.01
# tauValidate=(((tauValidate-mintaured)/(maxtaured-mintaured))*0.98)+0.01
# tauTrain=(((tauTrain-mintaured)/(maxtaured-mintaured))*0.98)+0.01



flux=np.exp(-1.*taured)
fluxTest = np.exp(-1.*tauTest)
fluxValidate = np.exp(-1.*tauValidate)
fluxTrain = np.exp(-1.*tauTrain)

signalRMS = np.std(flux)

rmsnoise=signalRMS/10 #this is the rms noise to add - if it's zero then we are try 
bignoise=signalRMS/5
hugenoise=signalRMS/2.5
# 1 is the mean of the normal distribution you are choosing from
# 2 is the standard deviation of the normal distribution
# 3 is the number of elements you get in array noise

#SCALING

# minflux=flux.min()
# maxflux=flux.max()
# flux=(((flux-minflux)/(maxflux-minflux))*0.98)+0.01
# fluxTest=(((fluxTest-minflux)/(maxflux-minflux))*0.98)+0.01
# fluxValidate=(((fluxValidate-minflux)/(maxflux-minflux))*0.98)+0.01
# fluxTrain=(((fluxTrain-minflux)/(maxflux-minflux))*0.98)+0.01

fluxRMS = np.std(flux)
print("fluxRMS", fluxRMS)

# noise = np.random.normal(0.0,rmsnoise,(nlos,npix))
# flux=flux+noise

# noiseTest = np.random.normal(0.0,rmsnoise,fluxTest.shape)
# bigNoiseTest = np.random.normal(0.0,bignoise,fluxTest.shape)
# hugeNoiseTest = np.random.normal(0.0,hugenoise,fluxTest.shape)

# bigNoiseFluxTest = fluxTest + bigNoiseTest
# hugeNoiseFluxTest = fluxTest + hugeNoiseTest
# fluxTest=fluxTest+noiseTest

# noiseValidate = np.random.normal(0.0,rmsnoise,fluxValidate.shape)
# bigNoiseValidate = np.random.normal(0.0,bignoise,fluxValidate.shape)
# hugeNoiseValidate = np.random.normal(0.0,hugenoise,fluxValidate.shape)

# bigNoiseFluxValidate = fluxValidate + bigNoiseValidate
# hugeNoiseFluxValidate = fluxValidate + hugeNoiseValidate
# fluxValidate=fluxValidate+noiseValidate

# noiseTrain = np.random.normal(0.0,rmsnoise,fluxTrain.shape)
# bigNoiseTrain = np.random.normal(0.0,bignoise,fluxTrain.shape)
# hugeNoiseTrain = np.random.normal(0.0,hugenoise,fluxTrain.shape)

# bigNoiseFluxTrain = fluxTrain + bigNoiseTrain
# hugeNoiseFluxTrain = fluxTrain + hugeNoiseTrain
# fluxTrain=fluxTrain+noiseTrain


taured = torch.from_numpy(taured)
tauTest = torch.from_numpy(tauTest)
tauValidate = torch.from_numpy(tauValidate)
tauTrain = torch.from_numpy(tauTrain)

flux = torch.from_numpy(flux)
fluxTest = torch.from_numpy(fluxTest)
fluxValidate = torch.from_numpy(fluxValidate)
fluxTrain = torch.from_numpy(fluxTrain)
# bigNoiseFluxTest = torch.from_numpy(bigNoiseFluxTest)
# hugeNoiseFluxTest = torch.from_numpy(hugeNoiseFluxTest)
# bigNoiseFluxTrain = torch.from_numpy(bigNoiseFluxTrain)
# hugeNoiseFluxTrain = torch.from_numpy(hugeNoiseFluxTrain)
# bigNoiseFluxValidate = torch.from_numpy(bigNoiseFluxValidate)
# hugeNoiseFluxValidate = torch.from_numpy(hugeNoiseFluxValidate)

# cleanFluxTest = torch.from_numpy(cleanFluxTest)
# cleanFluxTrain = torch.from_numpy(cleanFluxTrain)
# cleanFluxValidate = torch.from_numpy(cleanFluxValidate)

tauredzero=taured[0,0,...]

fluxzero=flux[0,0,...]

#orange is the flux and blue is the optical depth, both scaled to 0.01-0.99
plt.plot(tauredzero.numpy())
plt.plot(fluxzero.numpy())
plt.show()

NameError: name 'taured' is not defined

In [ ]:
# tauredTest = taured[480:,...]
# fluxTest = flux[480:,...]
# tauredTrain = taured[:480,...]
# fluxTrain = flux[:480,...]
# print(tauredTest.shape)
# print(fluxTest.shape)
# print(tauredTrain.shape)
# print(fluxTrain.shape)

In [17]:
def onePointTrainSplit(num):
    indices = torch.randint(low=0,high=tauTrain.shape[0],size=(num,))
    return tauTrain[indices,0,256], fluxTrain[indices,0,...]

def fullTrainSplit(num):
    indices = torch.randint(low=0,high=tauTrain.shape[0],size=(num,))
    return tauTrain[indices,0,...], fluxTrain[indices,0,...]

# def revolveTrainSplit(tauredSet, fluxSet, num):
#     indices = torch.randint(low=0,high=tauredSet.shape[0],size=(num,))
#     taus = torch.zeros((512*num))
#     fluxs = torch.zeros((512*num,512))
#     counter = 0
#     tempFlux = torch.zeros((1024,))
#     for i in range(num):
#         for midpoint in range(512):
#             taus[512*i + midpoint] = tauredSet[indices[i],midpoint]
#             tempFlux[0:512] = fluxSet[indices[i],...]
#             tempFlux[512:] = fluxSet[indices[i],...]
#             fluxs[512*i+midpoint,...] = tempFlux[midpoint:512+midpoint]
#     return taus, fluxs

def revolveTrainSplit(num):
    indices = torch.randint(low=0,high=tauTrain.shape[0],size=(num,))
    taus = torch.zeros((512*num))
    fluxs = torch.zeros((512*num,512))
    counter = 0
    taus = torch.flatten(tauTrain[indices])
    for i in range(num):
        for midpoint in range(512):
            fluxs[512*i+midpoint,...] = torch.cat((fluxTrain[indices[i],midpoint-256:],fluxTrain[indices[i],:midpoint-256]),dim=0)
    return taus, fluxs

noiseDict = {
    "none": 0,
    "low": 1/10,
    "mid": 1/5,
    "high": 1/2.5
}

def trueRandomRevolve(num, noise="low", source="train"):
    if source == "test":
        tSet = tauTest
        fSet = fluxTest
    elif source == "validate":
        tSet = tauValidate
        fSet = fluxValidate
    elif source == "train":
        tSet = tauTrain
        fSet = fluxTrain
    indices = torch.randint(low=0,high=tSet.shape[0],size=(num,))
    points = torch.randint(low=0,high=512,size=(num,))
    taus = torch.zeros((num))
    fluxs = torch.zeros(num,512)
    for i in range(num):
        fluxs[i] = spin(fSet[indices[i],0], points[i])
        taus[i] = tSet[indices[i],0,points[i]]
    fluxs += torch.from_numpy(np.random.normal(0.0,signalRMS*noiseDict[noise],fluxs.shape))
    return taus, fluxs

def validate(noise="low"):
    tSet = tauValidate
    fSet = fluxValidate
    size = len(torch.flatten(tSet))
    taus = torch.zeros((size*512))
    fluxs = torch.zeros((size*512, 512))
    for i in range(size*512):
        taus[i] = tSet[i//512,0,i%512]
        fluxs[i] = spin(fSet[i//512, 0], i%512)
    fluxs += torch.from_numpy(np.random.normal(0.0,signalRMS*noiseDict[noise],fluxs.shape))
    return taus, fluxs
    
def spin(tensor, midpoint):
    return torch.cat((tensor[midpoint-256:],tensor[:midpoint-256]),dim=0)

IndentationError: expected an indented block (<ipython-input-17-087df0587740>, line 42)

In [26]:
def spinTest():
    a = torch.arange(512)
    for i in range(512):
        assert(spin(a,i)[256] == i)
#         print("Passed test %d/512" %(i+1))
    print("The spin function works")
    
spinTest()

The spin function works
torch.Size([50])
torch.Size([50, 512])


One Point Convolutional Network

In [ ]:
class OnePointConvNet(torch.nn.Module):
    def __init__(self):
        super(OnePointConvNet, self).__init__()
        self.hidden1 = torch.nn.Conv1d(1, 3, 5)   # hidden layer
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.hidden2 = torch.nn.Linear(504,1000)
        self.hidden3 = torch.nn.Linear(1000, 2000)
        self.hidden4 = torch.nn.Linear(2000, 2500)
        self.hidden5 = torch.nn.Linear(2500,3200)
        self.hidden6 = torch.nn.Linear(3200,4800)
        self.hidden7 = torch.nn.Conv1d(3,1,11)
        self.predict = torch.nn.Linear(4790, 1)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.pool1(x)
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = F.relu(self.hidden5(x))
        x = F.relu(self.hidden6(x))
        x = F.relu(self.hidden7(x))
        x = self.predict(x)
        return x


Simple Regression (Formerly Linear Regression, was used to find slope)

In [ ]:
class SimpleConvNet(torch.nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.hidden1 = torch.nn.Conv1d(1, 2, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,768)
        self.lin2 = torch.nn.Linear(768,1024)
        self.hidden2 = torch.nn.Conv1d(2,1,1)
        self.predict = torch.nn.Linear(1024,512)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

In [ ]:
class SimpleMidpoint(torch.nn.Module):
    def __init__(self):
        super(SimpleMidpoint, self).__init__()
        self.hidden1 = torch.nn.Conv1d(1, 2, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,64)
        self.lin2 = torch.nn.Linear(64,8)
        self.hidden2 = torch.nn.Conv1d(2,1,1)
        self.predict = torch.nn.Linear(8,1)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.hidden2(x))
#         x = x.view((-1,1,16))
        x = self.predict(x)             # linear output
        return x

In [ ]:
class MoreConvolution1(torch.nn.Module):
    def __init__(self):
        super(MoreConvolution1, self).__init__()
        self.conv1 = torch.nn.Conv1d(1, 4, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,64)
        self.lin2 = torch.nn.Linear(64,8)
#         self.hidden2 = torch.nn.Conv1d(4,1,1)
        self.predict = torch.nn.Linear(32,1)   # output layer

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
#         x = F.relu(self.hidden2(x))
        x = x.view((-1,1,32))
        x = self.predict(x)             # linear output
        return x
    
class MoreConvolution2(torch.nn.Module):
    def __init__(self):
        super(MoreConvolution2, self).__init__()
        self.conv1 = torch.nn.Conv1d(1, 4, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,64)
        self.lin2 = torch.nn.Linear(64,8)
        self.predict = torch.nn.Linear(32,1)   # output layer

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

In [ ]:
class Deeper(torch.nn.Module):
    def __init__(self):
        super(Deeper, self).__init__()
        self.hidden1 = torch.nn.Conv1d(1, 4, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,128)
        self.lin2 = torch.nn.Linear(128,32)
        self.lin3 = torch.nn.Linear(32,4)
        self.lin4 = torch.nn.Linear(4,1)
        self.predict = torch.nn.Linear(4,1)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = x.view((-1,1,4))
        x = self.predict(x)             # linear output
        return x

class BigBrain(torch.nn.Module):
    def __init__(self):
        super(BigBrain, self).__init__()
        self.hidden1 = torch.nn.Conv1d(1, 4, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,64)
        self.lin2 = torch.nn.Linear(64,120)
        self.hidden2 = torch.nn.Linear(120,30)
        self.hidden3 = torch.nn.Linear(30,5)
        self.hidden4 = torch.nn.Linear(5, 1)
        self.predict = torch.nn.Linear(4,1)
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.pool1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.hidden2(x))
#         x = x.view((-1,1,16))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = x.view((-1,1,4))
        x = self.predict(x)             # linear output
        return x


In [16]:
def trueLog(a):
    x = []
    y = []
    n = len(a)
    for i in range(n):
        if a[i] > 0:
            x.append(i)
            y.append(-log(a[i]))
            
    func = CubicSpline(x,y)
    desiredXs = np.arange(n)
    result = func(desiredXs)
    return result


tensor([0.0000, 0.6931, 1.0986, 1.0986, 1.0986])


In [2]:
netDictionary = {
#     "OnePointConvNet": OnePointConvNet,     DEPRECATED
#     "SimpleConvNet": SimpleConvNet,         DEPRECATED
    "SimpleMidpoint": SimpleMidpoint,
    "MoreConvolution1": MoreConvolution1,
    "MoreConvolution2": MoreConvolution2,
    "Deeper": Deeper,
    "BigBrain": BigBrain
    
}



class NetFactory():
    def __init__(self, netType, printAll = True):
        self.netType = netType
        self.net = netDictionary[netType]()
        self.printAll = printAll
        
        
    #to test
    def onePointValidate(self):
        loss_func = torch.nn.MSELoss()
        
        loss = np.zeros(32)
        self.net.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for i in range(32):
                y = fluxTest[i,...].float()
                y = torch.reshape(y, (-1,1,512))
                prediction = self.net(y)
                loss[i] = loss_func(prediction, tauTest[i,0,256].float())
        self.net.train()
        return np.mean(loss)
    
    def fullTest(self, noise):
        loss_func = torch.nn.MSELoss()
        fluxs = torch.zeros((fluxTest.shape[0]*512,1,512))
        taus = torch.zeros((fluxTest.shape[0]*512,1,1))
        noiseGen =  torch.distributions.normal.Normal(0.0, signalRMS*noiseDict[noise], validate_args=None)
        for i in range(fluxTest.shape[0]):
            for midpoint in range(512):
                fluxs[512*i+midpoint,0] = spin(fluxTest[i,0], midpoint) + noiseGen.sample((512,)).view((1,1,512))
                taus[512*i+midpoint,0] = tauTest[i,0,midpoint]
        fluxs = fluxs.float()
        loss = torch.zeros(fluxTest.shape[0])
        self.net.eval()
        with torch.no_grad():
            for i in range(fluxTest.shape[0]):
                prediction = self.net(fluxs[512*i:512*(i+1)])
                loss[i] = loss_func(prediction, taus[512*i:512*(i+1)])
        self.net.train()
        return loss.mean()
    
     
    def randomRevolveRun(self, epochs, learningRate, noise="low"):
        now = datetime.now()
 
        print("now =", now)
        
        writer = SummaryWriter()
        
        writer.add_text("Net", str(self.net))
        writer.add_text("Epochs", str(epochs))
        writer.add_text("Learning Rate", str(learningRate))
        
        optimizer = torch.optim.Adam( self.net.parameters(), learningRate, weight_decay=0.0005 )
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=1)
        loss_func = torch.nn.MSELoss()
        
        allSame = 0
        
        for epoch in range(epochs):
            
            
            tauSet,fluxSet = trueRandomRevolve(1000, noise)
            tauSet = tauSet.float()
            fluxSet = fluxSet.float()
            tauSet = torch.reshape(tauSet, (-1,1,1))
            fluxSet = torch.reshape(fluxSet, (-1,1,512))
            
            prediction = self.net(fluxSet)
            if torch.std(prediction) < 0.00006:
                print("std = ", torch.std(prediction))
                print("They're all", prediction[1])
                allSame += 1
                if allSame > 50:
                    return 0
            else:
                allSame == 0
                
            
            
            loss = loss_func(prediction, tauSet)     # must be (1. nn output, 2. target)
            optimizer.zero_grad()   # clear gradients for next train
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
            scheduler.step()        # scheduler decreases learning rate geometrically every n epochs

            
            
            
            writer.add_scalar('Loss/train', loss, epoch)
            
            if epoch % 1000 == 0:
                hundredth = plt.figure()
                a = torch.flatten(tauSet)
                b = torch.flatten(prediction)
                plt.title("%dth epoch predictions" %epoch)
                plt.plot(a,"r", label="actual")
                plt.plot(b.detach().numpy(),"g.", label="prediction")
                plt.show()
                writer.add_figure("Every Hundred Epochs", hundredth)
                
                testLoss = self.fullTest(noise)
                writer.add_scalar('Loss/test', testLoss, epoch)
#             for i in self.net.__dict__['_modules']:
#                 if i[0:4] != "pool":
#                     writer.add_histogram(i + '/weight', getattr(self.net, i).weight.grad, epoch)
#                     writer.add_histogram(i + '/bias', getattr(self.net, i).bias.grad, epoch)
            
            if self.printAll:
                print("Epoch = ", epoch)
                print("Training Loss = ", loss)
                if epoch % 1000 == 0:
                    print("Test Loss = ", testLoss)
                    
            

#         a = tauSet.view((48,1))
#         b = fluxSet.view((48,512))
#         c = prediction.view((48,1))
#         testResults = plt.figure()
#         plt.plot(a.data.cpu().numpy(), "r", label="actual")
#         plt.title("prediction vs actual value")
#         plt.plot(c.data.cpu().numpy(), "g", label="prediction")
#         expPred = -np.log(b.data.cpu().numpy())
#         newExpPred = expPred[...,256]
#         plt.plot(newExpPred, 'b', label="exp")
#         plt.ylabel("Tau")
#         plt.xlabel("Sample #")
#         plt.legend()
#         plt.show()
        
        
#         writer.add_figure("Test Results", testResults)
        
#         taus, fluxs = trueRandomRevolve(100)
        taus = tauValidate[0,0,...]
        fluxs = torch.zeros((512,1,512))
        for i in range(512):
            fluxs[i] = spin(fluxValidate[0,0,...],i)
        taus = taus.float()
        fluxs = fluxs.float()
        taus = torch.reshape(taus, (-1,1,1))
        fluxs = torch.reshape(fluxs, (-1,1,512))
        prediction = self.net(fluxs)
        prediction = torch.flatten(prediction)
        taus = torch.flatten(taus)
        logPrediction = trueLog(torch.flatten(fluxs[256,0]))
                                       
        example = plt.figure()
        
        plt.plot(taus.data.cpu().numpy(), "r", label="actual")
        plt.plot(prediction.data.cpu().numpy(), "g", label="prediction")
        plt.plot(logPrediction, "b", label="log and cubic splines")
        plt.legend()
        plt.show()
        writer.add_figure("One Sightline", example)
        
        difference = plt.figure()
        plt.plot(prediction.data.cpu().numpy()-taus.data.cpu().numpy(), "g", label="prediction")
        plt.plot(logPrediction-taus.data.cpu().numpy(), "b", label="log and cubic splines")
        plt.legend()
        plt.title("Difference")
        plt.show()
        writer.add_figure("Difference", difference)
        
        fracDifference = plt.figure()
        plt.plot((prediction.data.cpu().numpy()-taus.data.cpu().numpy())/taus.data.cpu().numpy(), "g", label="prediction")
        plt.plot((logPrediction-taus.data.cpu().numpy())/taus.data.cpu().numpy(), "b", label="log and cubic splines")
        plt.legend()
        plt.title("Fractional Difference")
        plt.show()
        writer.add_figure("Fractional Difference", fracDifference)
        
#         taus, fluxs = validate(noise)
        taus, fluxs = trueRandomRevolve(10000, noise, source="validate")
        taus = taus.float()
        fluxs = fluxs.float()
        taus = torch.reshape(taus, (-1,1,1))
        fluxs = torch.reshape(fluxs, (-1,1,512))

        prediction = self.net(fluxs)
        
        a = torch.flatten(taus).detach().cpu().numpy()
        b = torch.flatten(prediction).detach().cpu().numpy()
        logP = trueLog(torch.flatten(fluxs))
        logP = logP[256::512]
        pearsonCoeff = pearsonr(a,b)
        if pearsonCoeff[0] != pearsonCoeff[0]:
            return 0
        print("Pearson's Coefficient is:")
        print(pearsonCoeff)
        print("Compared to log:")
        print(pearsonr(a,logP))
        
        c = np.zeros(a.shape)
        
        newA = np.where(a>1.5, a, c)
        newB = np.where(a>1.5, b, c)
        newLogP = np.where(a>1.5, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        highA = np.copy(newA)
        highB = np.copy(newB)
        newLogP = newLogP[np.nonzero(newLogP)]
        print("Pearson's Coefficient for high numbers")
        print(pearsonr(newA,newB))
        print("Compared to log:")
        print(pearsonr(newA,newLogP))
        
        
        print("percent high:")
        print(len(newA)/len(a))
        
        newA = np.where(a<1.5, a, c)
        newB = np.where(a<1.5, b, c)
        newLogP = np.where(a<1.5, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        newLogP = newLogP[np.nonzero(newLogP)]
        print("Pearson's Coefficient for low numbers")
        print(pearsonr(newA,newB))
        print("Compared to log:")
        print(pearsonr(newA,newLogP))
        
        print("percent low:")
        print(len(newA)/len(a))
        
        plt.figure()
        plt.scatter(a,b)
        plt.title("Scatter plot of Prediction vs Tau")
        plt.ylabel("Prediction")
        plt.xlabel("Tau")
        plt.show()
        
        plt.figure()
        plt.plot(highA[:100], "r", label="Tau")
        plt.plot(highB[:100], "g", label="Prediction")
        plt.title("High values of Tau vs Prediction")
        plt.legend()
        plt.show()
        
        
        
        return 1
        
    def averageLoss(self):
        
        loss_func = torch.nn.MSELoss()
        
        lossList = torch.zeros(100)
        self.net.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for epoch in range(100):
                tauSet,fluxSet = trueRandomRevolve(1000)
                tauSet = tauSet.float()
                fluxSet = fluxSet.float()
                tauSet = torch.reshape(tauSet, (-1,1,1))
                fluxSet = torch.reshape(fluxSet, (-1,1,512))

                prediction = self.net(fluxSet)

                loss = loss_func(prediction, tauSet)     # must be (1. nn output, 2. target)
                lossList[epoch] = loss
            
        self.net.train()
        return torch.mean(lossList)
    
    def judgement(self):
#         size = tauValidate.shape[0]*512
        size = 1000*512
        taus = torch.zeros((size))
        fluxs = torch.zeros(size,512)
        for i in range(size):
            sight = size//512
            midpoint = size % 512
            fluxs[i] = spin(fluxTrain[sight,0], midpoint)
            taus[i] = tauTrain[sight,0,midpoint]
        loss_func = torch.nn.MSELoss()
        
        self.net.eval()
        with torch.no_grad():
            
            taus = taus.float()
            fluxs = fluxs.float()
            taus = torch.reshape(taus, (-1,1,1))
            fluxs = torch.reshape(fluxs, (-1,1,512))

            prediction = self.net(fluxs)

            loss = loss_func(prediction, taus)     # must be (1. nn output, 2. target)
            a = torch.flatten(taus)
            b = torch.flatten(prediction)
            pearsonCoeff = pearsonr(a.detach().cpu().numpy(), b.detach().cpu().numpy())
        self.net.train()
        if pearsonCoeff[0] != pearsonCoeff[0]:
            return 0
        else:
            return pearsonCoeff[0]
        
    def hybridResults(self, noise="low", low = 0.8):
        print("Noise:")
        print(noise)
        print("Low:")
        print(low)
#         taus, fluxs = trueRandomRevolve(10000, noise, source="validate")
        taus, fluxs = trueRandomRevolve(1000, noise, source="validate")
        taus = taus.float()
        fluxs = fluxs.float()
        taus = torch.reshape(taus, (-1,1,1))
        fluxs = torch.reshape(fluxs, (-1,1,512))

        prediction = self.net(fluxs)
        
        a = torch.flatten(taus).detach().cpu().numpy()
        b = torch.flatten(prediction).detach().cpu().numpy()
        logP = trueLog(torch.flatten(fluxs))
        logP = logP[256::512]
        for i in range(len(b)):
            if logP[i] < low:
                b[i] = logP[i]
        pearsonCoeff = pearsonr(a,b)
        print("Pearson's Coefficient is:")
        print(pearsonCoeff)
        print("Compared to log:")
        print(pearsonr(a,logP))
        
        c = np.zeros(a.shape)
        
        newA = np.where(a>1.5, a, c)
        newB = np.where(a>1.5, b, c)
        newLogP = np.where(a>1.5, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        newLogP = newLogP[np.nonzero(newLogP)]
        print("Pearson's Coefficient for high numbers")
        print(pearsonr(newA,newB))
        print("Compared to log:")
        print(pearsonr(newA,newLogP))
        
        
        print("percent high:")
        print(len(newA)/len(a))
        
        newA = np.where(a<1.5, a, c)
        newB = np.where(a<1.5, b, c)
        newLogP = np.where(a<1.5, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        newLogP = newLogP[np.nonzero(newLogP)]
        print("Pearson's Coefficient for low numbers")
        print(pearsonr(newA,newB))
        print("Compared to log:")
        print(pearsonr(newA,newLogP))
        
        print("percent low:")
        print(len(newA)/len(a))

IndentationError: expected an indented block (<ipython-input-2-ee14053a9db2>, line 52)

In [37]:
def findIdealLearningRate(runs):
    epochs = 10000
    phi = (sqrt(5) + 1)/2
    print(phi)
#     x1 = log(0.0002)
#     x4 = log(0.002)
    x1 = log(0.000002)
    x4 = log(1)
    x2 = x1 + (x4 - x1)/phi
    x3 = x4 - (x4 - x1)/phi
    net1 = NetFactory("SimpleMidpoint", False)
    net2 = NetFactory("SimpleMidpoint", False)
    net3 = NetFactory("SimpleMidpoint", False)
    net4 = NetFactory("SimpleMidpoint", False)
    while net1.randomRevolveRun(epochs,exp(x1)) == 0:
        
        net1 = NetFactory("SimpleMidpoint", False)
        
    while net2.randomRevolveRun(epochs,exp(x2)) == 0:
        
        net2 = NetFactory("SimpleMidpoint", False)
        
    while net3.randomRevolveRun(epochs,exp(x3)) == 0:
        
        net3 = NetFactory("SimpleMidpoint", False)
        
    while net4.randomRevolveRun(epochs,exp(x4)) == 0:
        
        net4 = NetFactory("SimpleMidpoint", False)
    l1 = net1.judgement()
    l2 = net2.judgement()
    l3 = net3.judgement()
    l4 = net4.judgement()
    for i in range(runs):
        print("PEARSON COEFFICIENTS IN ORDER")
        print(l1, exp(x1))
        print(l2, exp(x2))
        print(l3, exp(x3))
        print(l4, exp(x4))
        if l2 > l3:
            x4 = x3
            x3 = x2
            newx = x1 + (x4 - x1)/phi
            newPos = "2"
        else:
            x1 = x2
            x2 = x3
            newx = x4 - (x4 - x1)/phi
            newPos = "3"
        newNet = NetFactory("SimpleMidpoint", False)
        while newNet.randomRevolveRun(epochs,exp(newx)) == 0:
            newNet = NetFactory("SimpleMidpoint", False)
        newLoss = newNet.judgement()
        if newPos == "2":
            x2 = newx
            l2 = newLoss
        elif newPos == "3":
            x3 = newx
            l3 = newLoss
    minLoss = min(l1,l2,l3,l4)
    print("The Ideal Learning Rate")
    if minLoss == l1:
        return exp(x1)
    elif minLoss == l2:
        return exp(x2)
    elif minLoss == l3:
        return exp(x3)
    else:
        return exp(x4)
        

1.618033988749895


NameError: name 'NetFactory' is not defined